#### Imports

In [2]:
import sys
import os

import numpy as np
import skimage.io

import tensorflow as tf

import keras.backend
import keras.callbacks
import keras.layers
import keras.models
import keras.optimizers

import utils.model_builder
import utils.data_provider
import utils.metrics
import utils.objectives
import utils.dirtools

from config import config_vars


Using TensorFlow backend.


#### Config

In [3]:
experiment_name = '01'

config_vars = utils.dirtools.setup_experiment(config_vars, experiment_name)

data_partitions = utils.dirtools.read_data_partitions(config_vars)

config_vars

{'root_directory': '/home/jupyter',
 'max_training_images': 0,
 'create_split_files': False,
 'training_fraction': 0.5,
 'validation_fraction': 0.25,
 'transform_images_to_PNG': True,
 'pixel_depth': 8,
 'min_nucleus_size': 25,
 'boundary_size': 2,
 'augment_images': False,
 'elastic_points': 16,
 'elastic_distortion': 5,
 'elastic_augmentations': 10,
 'learning_rate': 0.0001,
 'epochs': 15,
 'steps_per_epoch': 500,
 'batch_size': 10,
 'val_batch_size': 10,
 'rescale_labels': True,
 'crop_size': 256,
 'cell_min_size': 16,
 'boundary_boost_factor': 1,
 'object_dilation': 3,
 'raw_images_dir': '/home/jupyter/raw_images/',
 'raw_annotations_dir': '/home/jupyter/raw_annotations/',
 'path_files_training': '/home/jupyter/training.txt',
 'path_files_validation': '/home/jupyter/validation.txt',
 'path_files_test': '/home/jupyter/test.txt',
 'normalized_images_dir': '/home/jupyter/norm_images/',
 'boundary_labels_dir': '/home/jupyter/boundary_labels/',
 'experiment_dir': '/home/jupyter/experime

In [24]:
len(data_partitions2['validation'])

50

In [21]:
experiment_name = 'xx'

config_vars2 = utils.dirtools.setup_experiment(config_vars, experiment_name)

data_partitions2 = utils.dirtools.read_data_partitions(config_vars)

config_vars

{'root_directory': '/home/jupyter',
 'max_training_images': 0,
 'create_split_files': False,
 'training_fraction': 0.5,
 'validation_fraction': 0.25,
 'transform_images_to_PNG': True,
 'pixel_depth': 8,
 'min_nucleus_size': 25,
 'boundary_size': 2,
 'augment_images': False,
 'elastic_points': 16,
 'elastic_distortion': 5,
 'elastic_augmentations': 10,
 'learning_rate': 0.0001,
 'epochs': 15,
 'steps_per_epoch': 500,
 'batch_size': 10,
 'val_batch_size': 10,
 'rescale_labels': True,
 'crop_size': 256,
 'cell_min_size': 16,
 'boundary_boost_factor': 1,
 'object_dilation': 3,
 'raw_images_dir': '/home/jupyter/raw_images/',
 'raw_annotations_dir': '/home/jupyter/raw_annotations/',
 'path_files_training': '/home/jupyter/training.txt',
 'path_files_validation': '/home/jupyter/validation.txt',
 'path_files_test': '/home/jupyter/test.txt',
 'normalized_images_dir': '/home/jupyter/aitslab_images/reshaped_images/',
 'boundary_labels_dir': '/home/jupyter/aitslab_images/boundary_labels/reshaped_ma

#### Init data generators

In [6]:

# build session running on GPU 1
configuration = tf.compat.v1.ConfigProto()
configuration.gpu_options.allow_growth = True
configuration.gpu_options.visible_device_list = "0"
session = tf.compat.v1.Session(config = configuration)

# apply session
tf.compat.v1.keras.backend.set_session(session)

train_gen = utils.data_provider.random_sample_generator(
    config_vars["normalized_images_dir"],
    config_vars["boundary_labels_dir"],
    data_partitions["training"],
    config_vars["batch_size"],
    config_vars["pixel_depth"],
    config_vars["crop_size"],
    config_vars["crop_size"],
    config_vars["rescale_labels"]
)

val_gen = utils.data_provider.single_data_from_images(
     config_vars["normalized_images_dir"],
     config_vars["boundary_labels_dir"],
     data_partitions["validation"],
     config_vars["val_batch_size"],
     config_vars["pixel_depth"],
     config_vars["crop_size"],
     config_vars["crop_size"],
     config_vars["rescale_labels"]
)

#### Build model

In [7]:
# build model
model = utils.model_builder.get_model_3_class(config_vars["crop_size"], config_vars["crop_size"], activation=None)
model.summary()

#loss = "categorical_crossentropy"
loss = utils.objectives.weighted_crossentropy

metrics = [keras.metrics.categorical_accuracy, 
           utils.metrics.channel_recall(channel=0, name="background_recall"), 
           utils.metrics.channel_precision(channel=0, name="background_precision"),
           utils.metrics.channel_recall(channel=1, name="interior_recall"), 
           utils.metrics.channel_precision(channel=1, name="interior_precision"),
           utils.metrics.channel_recall(channel=2, name="boundary_recall"), 
           utils.metrics.channel_precision(channel=2, name="boundary_precision"),
          ]

optimizer = keras.optimizers.RMSprop(lr=config_vars["learning_rate"])

model.compile(loss=loss, metrics=metrics, optimizer=optimizer)

# Performance logging
callback_csv = keras.callbacks.CSVLogger(filename=config_vars["csv_log_file"])

callbacks=[callback_csv]

/home/jupyter/utils/model_builder.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(x)
/home/jupyter/utils/model_builder.py:17: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9)`
  a = keras.layers.BatchNormalization(**option_dict_bn)(a)
/home/jupyter/utils/model_builder.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(a)
/home/jupyter/utils/model_builder.py:20: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9)`
  a = keras.layers.BatchNormalization(**option_dict_bn)(a)
/home/jupyter/utils/model_builder.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", padd

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [8]:
# TRAIN
statistics = model.fit_generator(
    generator=train_gen,
    steps_per_epoch=config_vars["steps_per_epoch"],
    #epochs=config_vars["epochs"],
    epochs = 5,
    validation_data=val_gen,
    validation_steps=int(len(data_partitions["validation"])/config_vars["val_batch_size"]),
    callbacks=callbacks,
    verbose = 1
)

model.save_weights(config_vars["model_file"])

print('Done! :)')

Training with 100 images.
Epoch 1/5
500/500 [==============================] - 783s 2s/step - loss: 0.2506 - categorical_accuracy: 0.9451 - background_recall: 0.9625 - background_precision: 0.9969 - interior_recall: 0.8680 - interior_precision: 0.9820 - boundary_recall: 0.9477 - boundary_precision: 0.4970 - val_loss: 0.1895 - val_categorical_accuracy: 0.9604 - val_background_recall: 0.9753 - val_background_precision: 0.9984 - val_interior_recall: 0.8898 - val_interior_precision: 0.9893 - val_boundary_recall: 0.9579 - val_boundary_precision: 0.5478
Epoch 2/5
500/500 [==============================] - 770s 2s/step - loss: 0.1789 - categorical_accuracy: 0.9561 - background_recall: 0.9725 - background_precision: 0.9981 - interior_recall: 0.8848 - interior_precision: 0.9925 - boundary_recall: 0.9551 - boundary_precision: 0.5375 - val_loss: 0.1724 - val_categorical_accuracy: 0.9559 - val_background_recall: 0.9654 - val_background_precision: 0.9991 - val_interior_recall: 0.9100 - val_interior

## Train on manually annotated images (w/ affine transforms)

#### config

In [2]:
experiment_name = 'MA_1' #manually annotated 1 (affine transformations + flip + rotate + illumination)

config_vars = utils.dirtools.setup_experiment(config_vars, experiment_name)

data_partitions = utils.dirtools.read_data_partitions(config_vars)

config_vars

{'root_directory': '/home/jupyter',
 'max_training_images': 0,
 'create_split_files': False,
 'training_fraction': 0.5,
 'validation_fraction': 0.25,
 'transform_images_to_PNG': True,
 'pixel_depth': 8,
 'min_nucleus_size': 25,
 'boundary_size': 2,
 'augment_images': False,
 'elastic_points': 16,
 'elastic_distortion': 5,
 'elastic_augmentations': 10,
 'learning_rate': 0.0001,
 'epochs': 15,
 'steps_per_epoch': 500,
 'batch_size': 10,
 'val_batch_size': 10,
 'rescale_labels': True,
 'crop_size': 256,
 'cell_min_size': 16,
 'boundary_boost_factor': 1,
 'object_dilation': 3,
 'raw_images_dir': '/home/jupyter/raw_images/',
 'raw_annotations_dir': '/home/jupyter/raw_annotations/',
 'path_files_training': '/home/jupyter/training.txt',
 'path_files_validation': '/home/jupyter/validation.txt',
 'path_files_test': '/home/jupyter/test.txt',
 'normalized_images_dir': '/home/jupyter/norm_images/',
 'boundary_labels_dir': '/home/jupyter/boundary_labels/',
 'experiment_dir': '/home/jupyter/experime

In [3]:
config_vars['normalized_images_dir'] = '/home/jupyter/aitslab_images/reshaped_images/'
config_vars['boundary_labels_dir'] = '/home/jupyter/aitslab_images/boundary_labels/reshaped_manually_annotated/'

In [4]:
filelist = sorted(os.listdir('/home/jupyter/aitslab_images/raw_annotations'))
filelist.remove('.ipynb_checkpoints')

In [5]:
data_partitions['training'] = filelist[0:10]
data_partitions['validation'] = filelist[10:]

In [6]:
# build session running on GPU 1
configuration = tf.compat.v1.ConfigProto()
configuration.gpu_options.allow_growth = True
configuration.gpu_options.visible_device_list = "0"
session = tf.compat.v1.Session(config = configuration)

# apply session
tf.compat.v1.keras.backend.set_session(session)

train_gen = utils.data_provider.random_sample_generator(
    config_vars["normalized_images_dir"],
    config_vars["boundary_labels_dir"],
    data_partitions["training"],
    config_vars["batch_size"],
    config_vars["pixel_depth"],
    config_vars["crop_size"],
    config_vars["crop_size"],
    config_vars["rescale_labels"]
)

val_gen = utils.data_provider.single_data_from_images(
     config_vars["normalized_images_dir"],
     config_vars["boundary_labels_dir"],
     data_partitions["validation"],
     config_vars["val_batch_size"],
     config_vars["pixel_depth"],
     config_vars["crop_size"],
     config_vars["crop_size"],
     config_vars["rescale_labels"]
)

In [7]:
# build model
model = utils.model_builder.get_model_3_class(config_vars["crop_size"], config_vars["crop_size"], activation=None)
model.load_weights('/home/jupyter/experiments/01/model.hdf5')
model.summary()

#loss = "categorical_crossentropy"
loss = utils.objectives.weighted_crossentropy

metrics = [keras.metrics.categorical_accuracy, 
           utils.metrics.channel_recall(channel=0, name="background_recall"), 
           utils.metrics.channel_precision(channel=0, name="background_precision"),
           utils.metrics.channel_recall(channel=1, name="interior_recall"), 
           utils.metrics.channel_precision(channel=1, name="interior_precision"),
           utils.metrics.channel_recall(channel=2, name="boundary_recall"), 
           utils.metrics.channel_precision(channel=2, name="boundary_precision"),
          ]

optimizer = keras.optimizers.RMSprop(lr=config_vars["learning_rate"])

model.compile(loss=loss, metrics=metrics, optimizer=optimizer)

# Performance logging
callback_csv = keras.callbacks.CSVLogger(filename=config_vars["csv_log_file"])

callbacks=[callback_csv]

/home/jupyter/utils/model_builder.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(x)
/home/jupyter/utils/model_builder.py:17: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9)`
  a = keras.layers.BatchNormalization(**option_dict_bn)(a)
/home/jupyter/utils/model_builder.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(a)
/home/jupyter/utils/model_builder.py:20: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9)`
  a = keras.layers.BatchNormalization(**option_dict_bn)(a)
/home/jupyter/utils/model_builder.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", padd

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [8]:
# TRAIN
statistics = model.fit_generator(
    generator=train_gen,
    steps_per_epoch=config_vars["steps_per_epoch"],
    #epochs=config_vars["epochs"],
    epochs = 2,
    validation_data=val_gen,
    validation_steps=,
    callbacks=callbacks,
    verbose = 1
)

print('Done! :)')

Training with 10 images.
Epoch 1/2
500/500 [==============================] - 774s 2s/step - loss: 0.1275 - categorical_accuracy: 0.9866 - background_recall: 0.9901 - background_precision: 0.9996 - interior_recall: 0.9037 - interior_precision: 0.9468 - boundary_recall: 0.9179 - boundary_precision: 0.4661 - val_loss: 0.0733 - val_categorical_accuracy: 0.9917 - val_background_recall: 0.9935 - val_background_precision: 1.0000 - val_interior_recall: 0.9536 - val_interior_precision: 0.9570 - val_boundary_recall: 0.8738 - val_boundary_precision: 0.4582
Epoch 2/2
500/500 [==============================] - 758s 2s/step - loss: 0.0746 - categorical_accuracy: 0.9925 - background_recall: 0.9953 - background_precision: 0.9997 - interior_recall: 0.9236 - interior_precision: 0.9864 - boundary_recall: 0.9395 - boundary_precision: 0.5596 - val_loss: 0.0597 - val_categorical_accuracy: 0.9939 - val_background_recall: 0.9957 - val_background_precision: 0.9999 - val_interior_recall: 0.9517 - val_interior_

In [9]:
config_vars["model_file"]

'/home/jupyter/experiments/MA_1/model.hdf5'

In [10]:
model.save_weights(config_vars["model_file"])

### Train on manually annotaded images (W/O affine transforms)

In [3]:
experiment_name = 'MA_3' #manually annotated 3(flip + rotate + illumination)

config_vars = utils.dirtools.setup_experiment(config_vars, experiment_name)

data_partitions = utils.dirtools.read_data_partitions(config_vars)

#config_vars

In [4]:
config_vars['normalized_images_dir'] = '/home/jupyter/aitslab_images/reshaped_images/'
config_vars['boundary_labels_dir'] = '/home/jupyter/aitslab_images/boundary_labels/reshaped_manually_annotated/'

In [5]:
filelist = sorted(os.listdir('/home/jupyter/aitslab_images/raw_annotations'))
filelist.remove('.ipynb_checkpoints')

In [8]:
filelist = [f for f in filelist if len(f)==35]

In [9]:
filelist

['MFGTMPcx7_170803210001_A01f29d0.png',
 'MFGTMPcx7_170803210001_A05f27d0.png',
 'MFGTMPcx7_170803210001_B18f27d0.png',
 'MFGTMPcx7_170803210001_B19f26d0.png',
 'MFGTMPcx7_170803210001_J12f29d0.png']

In [10]:
data_partitions['training'] = filelist[0:3]
data_partitions['validation'] = filelist[3:]

In [12]:
# build session running on GPU 1
configuration = tf.compat.v1.ConfigProto()
configuration.gpu_options.allow_growth = True
configuration.gpu_options.visible_device_list = "0"
session = tf.compat.v1.Session(config = configuration)

# apply session
tf.compat.v1.keras.backend.set_session(session)

train_gen = utils.data_provider.random_sample_generator(
    config_vars["normalized_images_dir"],
    config_vars["boundary_labels_dir"],
    data_partitions["training"],
    config_vars["batch_size"],
    config_vars["pixel_depth"],
    config_vars["crop_size"],
    config_vars["crop_size"],
    config_vars["rescale_labels"]
)

val_gen = utils.data_provider.single_data_from_images(
     config_vars["normalized_images_dir"],
     config_vars["boundary_labels_dir"],
     data_partitions["validation"],
     config_vars["val_batch_size"],
     config_vars["pixel_depth"],
     config_vars["crop_size"],
     config_vars["crop_size"],
     config_vars["rescale_labels"]
)

In [13]:
# build model
model = utils.model_builder.get_model_3_class(config_vars["crop_size"], config_vars["crop_size"], activation=None)
model.load_weights('/home/jupyter/experiments/01/model.hdf5')
#model.summary()

#loss = "categorical_crossentropy"
loss = utils.objectives.weighted_crossentropy

metrics = [keras.metrics.categorical_accuracy, 
           utils.metrics.channel_recall(channel=0, name="background_recall"), 
           utils.metrics.channel_precision(channel=0, name="background_precision"),
           utils.metrics.channel_recall(channel=1, name="interior_recall"), 
           utils.metrics.channel_precision(channel=1, name="interior_precision"),
           utils.metrics.channel_recall(channel=2, name="boundary_recall"), 
           utils.metrics.channel_precision(channel=2, name="boundary_precision"),
          ]

optimizer = keras.optimizers.RMSprop(lr=config_vars["learning_rate"])

model.compile(loss=loss, metrics=metrics, optimizer=optimizer)

# Performance logging
callback_csv = keras.callbacks.CSVLogger(filename=config_vars["csv_log_file"])

callbacks=[callback_csv]

/home/jupyter/utils/model_builder.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(x)
/home/jupyter/utils/model_builder.py:17: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9)`
  a = keras.layers.BatchNormalization(**option_dict_bn)(a)
/home/jupyter/utils/model_builder.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  a = keras.layers.Convolution2D(64, 3, 3, **option_dict_conv)(a)
/home/jupyter/utils/model_builder.py:20: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(momentum=0.9)`
  a = keras.layers.BatchNormalization(**option_dict_bn)(a)
/home/jupyter/utils/model_builder.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", padd

In [21]:
# TRAIN
statistics = model.fit_generator(
    generator=train_gen,
    steps_per_epoch=config_vars["steps_per_epoch"],
    #epochs=config_vars["epochs"],
    epochs = 2,
    validation_data=val_gen,
    validation_steps=0.2,
    callbacks=callbacks,
    verbose = 1
)

print('Done! :)')

Epoch 1/2
 61/500 [==>...........................] - ETA: 11:05 - loss: 0.0650 - categorical_accuracy: 0.9927 - background_recall: 0.9955 - background_precision: 0.9997 - interior_recall: 0.9313 - interior_precision: 0.9883 - boundary_recall: 0.9063 - boundary_precision: 0.4498

KeyboardInterrupt: 

In [1]:
# TRAIN
statistics = model.fit_generator(
    generator=train_gen,
    steps_per_epoch=20,
    #epochs=config_vars["epochs"],
    epochs = 5,
    validation_data=val_gen,
    validation_steps=0.2,
    callbacks=callbacks,
    verbose = 1
)

print('Done! :)')

NameError: name 'model' is not defined

In [24]:
statistics.history

{'val_loss': [0.09362031519412994],
 'val_categorical_accuracy': [0.9894332885742188],
 'val_background_recall': [0.9953933954238892],
 'val_background_precision': [0.9989851117134094],
 'val_interior_recall': [0.874977171421051],
 'val_interior_precision': [0.9904820919036865],
 'val_boundary_recall': [0.8804903030395508],
 'val_boundary_precision': [0.41362762451171875],
 'loss': [0.06180456504225731],
 'categorical_accuracy': [0.99327147],
 'background_recall': [0.995819],
 'background_precision': [0.99984866],
 'interior_recall': [0.9330703],
 'interior_precision': [0.9867552],
 'boundary_recall': [0.9131199],
 'boundary_precision': [0.43902978]}

In [25]:
model.save_weights(config_vars["model_file"])